In [2]:

import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from gensim.models import Phrases,phrases
from gensim.models import Word2Vec
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from gensim.scripts import word2vec2tensor
from itertools import combinations

In [ ]:
folder = '/Users/amycweng/Digital Humanities/charityTXT'
# folder = '/Users/amycweng/Digital Humanities/playsTXT'

# for perkins texts
# latin = ['A54390','A09452','A09447','A09349']
# welsh = ['A90503','A90506']
def getTexts(folder):
    texts = {}
    for file in os.listdir(folder):
        if 'NOTES' not in file: 
            name = file.split('.')[0]
            path = os.path.join(folder,file)
            f = open(path,'r')
            data = f.readlines()[0]
            texts[name] = data
            f.close()
    return texts


In [ ]:
all_texts = getTexts(folder)
tokenized = []
for text in all_texts.values():
    words = []
    text = text.split(' ')
    for t in text:
        if len(t) != 0: words.append(t)
    tokenized.append(words)
print(len(tokenized))

In [ ]:
# Creared word2vec model and store it
word2vec = Word2Vec(tokenized, min_count=2,sg=1)
# word2vec.wv.save_word2vec_format('/srv/data/tensor/sermons.model')
# word2vec.save('perkins.model')
word2vec.save('/Users/amycweng/Digital Humanities/Early-Modern-London/Other_Files/charity.model')

In [ ]:
words = ['charity','charitable','alm','bequest',
        'poor','rate','usury','usurer','give','bequeath',
        'help','money','beneficence','poverty','credit',
        'creditor','loan','lend','new','novel','novelty',
        'wealth','profit','profitable','beneficial',
        'commodity','thrift','thrifty','industry']

for term in words: 
    if term in word2vec.wv: print(f'{term}: {word2vec.wv.most_similar(term,topn=5)}\n')


In [14]:
words = ['citizen']
sermons_model = Word2Vec.load('/Users/amycweng/Digital Humanities/Early-Modern-London/Other_Files/charity.model')
play_model = Word2Vec.load('/Users/amycweng/Digital Humanities/Early-Modern-London/Other_Files/plays.model')
for term in words: 
    if term in sermons_model.wv: print(f'sermons {term}: {sermons_model.wv.most_similar(term,topn=10)}\n')
    if term in play_model.wv: print(f'plays {term}: {play_model.wv.most_similar(term,topn=10)}\n')


sermons citizen: [('livery', 0.8035364151000977), ('holiday', 0.7944857478141785), ('depasture', 0.7907779812812805), ('yesterday', 0.7875796556472778), ('pilgrim', 0.7871444225311279), ('talent', 0.7870103716850281), ('hive', 0.7782825231552124), ('cheese', 0.7765907645225525), ('britain', 0.7763808369636536), ('epicure', 0.7750282287597656)]

plays citizen: [('courtier', 0.9322307109832764), ('proper', 0.9166803956031799), ('fool', 0.9080363512039185), ('silly', 0.9075518846511841), ('coxcomb', 0.9068408608436584), ('beggar', 0.9066614508628845), ('cuckold', 0.9055889248847961), ('prentice', 0.9034937024116516), ('english', 0.9027708768844604), ('rare', 0.8978953957557678)]



In [ ]:
#Create PCA table
pca = PCA(n_components=2)
pca_results = pca.fit_transform(word2vec.wv.get_normed_vectors())
pca_df = pd.DataFrame(pca_results, index=word2vec.wv.key_to_index, columns=["pc1","pc2"])


In [ ]:
# Create PCA plots
pca_df.plot(x='pc1',y='pc2',kind="scatter",figsize=(15, 10))

In [ ]:
# Create tsv file for word2vec model
word2vec2tensor.word2vec2tensor('/srv/data/tensor/relevantep.model', '/srv/data/tensor/TSV/relevantep', binary=False)

In [ ]:
# Scatter plot with label
ax = pca_df.plot(x='pc1',y='pc2',kind="scatter",figsize=(15, 10),alpha=0)
for txt in pca_df.index:
    if txt in ['charity','charities','talent','talents','profit','profits']:
        x = pca_df.pc1.loc[txt]
        y = pca_df.pc2.loc[txt]
        ax.annotate(txt, (x,y))
plt.show()

In [ ]:
#Heatmap

def cosine_similarity(pair):

    '''
    Word2Vec.wv.similarity(word1,word2)
    '''
    word1, word2 = pair
    if word1 and word2 in word2vec.wv:
        sim = word2vec.wv.similarity(word1,word2)
        return sim
    return 0


def generate_heatmap_matrix(lexicon):

    pairs = list(combinations(lexicon, 2))
    sim_scores = [cosine_similarity(pair) for pair in pairs if cosine_similarity(pair) != 0]

    sim_df = pd.DataFrame({'pair': pairs, 'similarity': sim_scores})

    sim_df['word1'] = sim_df['pair'].apply(lambda x: lexicon.index(x[0]))
    sim_df['word2'] = sim_df['pair'].apply(lambda x: lexicon.index(x[1]))

    sim_df['pair'] = list(zip(sim_df.word1, sim_df.word2, sim_df.similarity))

    df_hm = pd.DataFrame({'word1': range(len(lexicon)),
                          'word2': range(len(lexicon)),
                          'similarity': pd.Series(np.ones(len(lexicon)))})

    df_hm = df_hm.pivot(index='word1', columns='word2').fillna(0)

    for row, col, similarity in sim_df.pair:

        df_hm.iloc[col,row] = similarity

    return df_hm


def plot_heatmap(lexicon,title):

    df_hm = generate_heatmap_matrix(lexicon)

    mask = np.zeros_like(df_hm)
    mask[np.triu_indices_from(mask)] = True
    plt.figure(figsize=(10, 8))
    sns.heatmap(df_hm,
                mask = mask,
                xticklabels = lexicon,
                yticklabels = lexicon,
                cmap="YlGnBu",
                )
    plt.xlabel('word 1')
    plt.ylabel('word 2')
    plt.title(title)
    plt.show()


In [ ]:
words = ['charity','charitable','alm','bequest',
        'poor','rate','usury','usurer','give','bequeath',
        'help','money','beneficence','poverty','credit',
        'creditor','loan','lend','new','novel','novelty',
        'wealth','profit','profitable','beneficial',
        'commodity','thrift','thrifty','industry']

plot_heatmap(words,'Similarity Between Key Word-Pairs in Charity Texts')